In [ ]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Input
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import numpy as np
from PIL import Image
import zipfile
import os
from sklearn.utils import shuffle
from time import sleep
# import pyautogui

In [ ]:
path = '/content/drive/My Drive/Licenta/classification/'
path_models = path + 'models+results3/'
path_dataset = path + 'dataset/'
path_images = path_dataset + 'images/'
path_np_arrays = path_dataset + 'np_arrays/'

Function that creates npz file

In [ ]:
def archive2np(name_load, name_save, path_images, path_np_arrays):
  z = zipfile.ZipFile(path_images + name_load)
  x = []
  for i, el in enumerate(z.infolist()):
    z.extract(el.filename, path_images)
    path_img = path_images + el.filename
    if path_img != '/content/drive/My Drive/Licenta/classification/dataset/images/111.png':   # problem with image 111.png
      im = Image.open(path_img)
      im_resized = im.resize((176, 216), Image.NEAREST)
      arr = np.asarray(im_resized)
      arr = np.expand_dims(arr, axis=2)
      x.append(arr)
      os.remove(path_img)
  x = np.array(x)
  np.savez(path_np_arrays + name_save, x=x)

Create npz files. No need to run the cell below again

In [ ]:
name_train_l_load = 'train_landscapes_yt_256x256_bw.zip'
name_train_l_save = 'train_landscapes_yt_216x176_bw'
name_train_p_load = 'train_portraits_celeba_216x176_bw.zip'
name_train_p_save = 'train_portraits_celeba_216x176_bw'

name_test_l_load = 'test_landscapes_yt_256x256_bw.zip'
name_test_l_save = 'test_landscapes_yt_216x176_bw'
name_test_p_load = 'test_portraits_celeba_216x176_bw.zip'
name_test_p_save = 'test_portraits_celeba_216x176_bw'

In [ ]:
archive2np(name_load=name_train_l_load, name_save=name_train_l_save, path_images=path_images, path_np_arrays=path_np_arrays)
archive2np(name_load=name_train_p_load, name_save=name_train_p_save, path_images=path_images, path_np_arrays=path_np_arrays)
archive2np(name_load=name_test_l_load, name_save=name_test_l_save, path_images=path_images, path_np_arrays=path_np_arrays)
archive2np(name_load=name_test_p_load, name_save=name_test_p_save, path_images=path_images, path_np_arrays=path_np_arrays)

Create x_train and y_train

In [ ]:
x_train_l = np.load(path_np_arrays + 'train_landscapes_yt_216x176_bw.npz')['x']
x_train_p = np.load(path_np_arrays + 'train_portraits_celeba_216x176_bw.npz')['x']

y_train_l = np.zeros(3635)
y_train_p = np.ones(3635)

In [ ]:
x_test_l = np.load(path_np_arrays + 'test_landscapes_yt_216x176_bw.npz')['x']
x_test_p = np.load(path_np_arrays + 'test_portraits_celeba_216x176_bw.npz')['x']

In [ ]:
x_train = np.append(x_train_l, x_train_p, axis=0)
y_train = np.append(y_train_l, y_train_p)

In [ ]:
inp = Input((216, 176, 1))
l = Conv2D(32, (3, 3), strides=(2,2), activation='relu', padding='same')(inp)
l = MaxPooling2D((2, 2))(l)
l = Dropout(0.2)(l)
l = Conv2D(64, (3, 3), strides=(2,2), activation='relu', padding='same')(l)
l = Dropout(0.2)(l)
l = Flatten()(l)
l = Dense(128, activation='relu')(l)
l = Dropout(0.2)(l)
l = Dense(128, activation='relu')(l)
l = Dropout(0.2)(l)
l = Dense(128, activation='relu')(l)
l = Dense(1, activation='sigmoid')(l)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Create and compile model

Train model

In [ ]:
model_checkpoint = ModelCheckpoint(path_models + 'weights.{epoch:02d}-{loss:.2f}.h5', monitor='loss', period=5)
model_reduce_lr = ReduceLROnPlateau(monitor='loss')
model.fit(x_train, y_train, epochs=100, batch_size=32, callbacks=[model_checkpoint, model_reduce_lr])

Test model

In [ ]:
model = load_model(path_models + 'weights.90-0.00.h5')

In [ ]:
y_pred_p = model.predict(x_test_p)
print(np.mean(y_pred_p))
count = 0
for y in y_pred_p:
  if y < 0.5:
    count += 1
print(count)

0.9939072
2


In [ ]:
y_pred_l = model.predict(x_test_l)
print(np.mean(y_pred_l))
# print(y_pred_l)
count = 0
for y in y_pred_l:
  if y > 0.5:
    count += 1
print(count)

3.779015e-05
0
